# Debugging Models

How to debug the training loop and how to ask the community for help

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 09/01/2026   | Martin | Created   | Notebook created for debugging LLM models | 
| 13/01/2026   | Martin | Update   | Debugging missing files from cloned repos | 

# Content

* [Introduction](#introduction)
* [Debugging Repo Files](#debugging-repo-files)

# Introduction

- What to do when you get an error
- How to ask for help on forums
- How to debug the training pipeline
- How to write good issues

# Debugging Repo Files

In [ ]:
from distutils.dir_util import copy_tree
from huggingface_hub import (
  notebook_login,
  Repository,
  snapshot_download,
  create_repo,
  get_full_repo_name,
  list_repo_files
)
from transformers import AutoConfig

In [5]:
notebook_login()

In [7]:
# Creating a repository
def copy_repository_template():
  # Clone the repo and extract local path
  template_repo_id = "lewtun/distilbert-base-uncased-finetuned-squad-d5716d28"
  commit_hash = "be3eaffc28669d7932492681cd5f3e8905e358b4"
  template_repo_dir = snapshot_download(template_repo_id, revision=commit_hash)

  # Create empty repo on the hub
  model_name = template_repo_id.split("/")[1]
  create_repo(model_name, exist_ok=True)

  # Clone empty repo
  new_repo_id = get_full_repo_name(model_name)
  new_repo_dir = model_name
  repo = Repository(local_dir=new_repo_dir, clone_from=new_repo_id)

  # Copy files
  copy_tree(template_repo_dir, new_repo_dir)

  # Push to hub
  repo.push_to_hub()

copy_repository_template()

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

/mnt/d/Software/venv/py310_ubun_venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Minimartzz/distilbert-base-uncased-finetuned-squad-d5716d28 into local empty directory.


Upload file pytorch_model.bin:   0%|          | 1.00/253M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/2.05k [00:00<?, ?B/s]

To https://huggingface.co/Minimartzz/distilbert-base-uncased-finetuned-squad-d5716d28
   d974578..c0ac1a5  main -> main



In [10]:
# List all files inside the repository
model_checkpoint = get_full_repo_name("distilbert-base-uncased-finetuned-squad-d5716d28")
list_repo_files(repo_id=model_checkpoint)

['.gitattributes',
 'README.md',
 'pytorch_model.bin',
 'special_tokens_map.json',
 'tokenizer_config.json',
 'training_args.bin',
 'vocab.txt']

In [14]:
# Missing config file, so get from the original model to push to repo
pretrained_config = "distilbert-base-uncased"
config = AutoConfig.from_pretrained(pretrained_config)

config.push_to_hub(model_checkpoint, commit_message="Add config.json")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Minimartzz/distilbert-base-uncased-finetuned-squad-d5716d28/commit/e711a54e18cc9f379c9d9d37ae52ad20e7a99027', commit_message='Add config.json', commit_description='', oid='e711a54e18cc9f379c9d9d37ae52ad20e7a99027', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Minimartzz/distilbert-base-uncased-finetuned-squad-d5716d28', endpoint='https://huggingface.co', repo_type='model', repo_id='Minimartzz/distilbert-base-uncased-finetuned-squad-d5716d28'), pr_revision=None, pr_num=None)

In [15]:
# Check again
list_repo_files(repo_id=model_checkpoint)

['.gitattributes',
 'README.md',
 'config.json',
 'pytorch_model.bin',
 'special_tokens_map.json',
 'tokenizer_config.json',
 'training_args.bin',
 'vocab.txt']

In [ ]:
reader = pipeline("question-answering", model=model_checkpoint, revision="main")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text
given a question. An example of a question answering dataset is the SQuAD
dataset, which is entirely based on that task. If you would like to fine-tune a
model on a SQuAD task, you may leverage the
examples/pytorch/question-answering/run_squad.py script.

🤗 Transformers is interoperable with the PyTorch, TensorFlow, and JAX
frameworks, so you can use your favourite tools for a wide variety of tasks!
"""

question = "What is extractive question answering?"
reader(question=question, context=context)

In [ ]:
%load_ext watermark
%watermark